In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle

# Import Feature Engineered Sales Transaction file
df = pd.read_csv('olist_processado.csv')

In [2]:
df.head(2)

,customer_id,customer_city,customer_state,order_id,order_purchase_timestamp,order_item_id,product_id,price,review_id,review_score,customer_lat,customer_long,product_category_name,quantidade_item
0,06b8999e2fba1a1fbc88172c00ba8bc7,franca,SP,00e7ee1b050b8499577073aeb2a297a1,2017-05-16 15:05:35,1,a9516a079e37a9c9c36b9b78b10169e8,124.99,88b8b52d46df026a9d1ad2136a59b30b,4,-20.500253,-47.400367,moveis_escritorio,1
1,8912fc0c3bbf1e2fbf35819e21706718,santarem,PA,c1d2b34febe9cd269e378117d6681172,2017-11-09 00:50:13,1,a9516a079e37a9c9c36b9b78b10169e8,112.99,7fc63200f12eebb5f387856afdd63db8,1,-2.441535,-54.720801,moveis_escritorio,2


## Top Categorias Vendidas

In [3]:
# Encontrando o número de unidades vendidas de cada categoria de produto
top_cliente_itens_df = df.groupby(['customer_id','product_category_name', 'customer_city', 'customer_state']).agg({'quantidade_item':'sum'})

top_cliente_itens_df.reset_index(inplace=True)

# Rank das categorias pela quantidade mais vendida, no nível do cliente

cliente_col = top_cliente_itens_df['customer_id']
quantidade_col = top_cliente_itens_df['quantidade_item'].astype(str)
top_cliente_itens_df['rank_top_vendidos'] = (cliente_col + quantidade_col).rank(method='min',ascending=False).astype(int)

# Lista os top 20 
top_cliente_itens_df.sort_values('rank_top_vendidos',ascending=True).head(20)

,customer_id,product_category_name,customer_city,customer_state,quantidade_item,rank_top_vendidos
98018,ffffa3172527f765de70084a7e53aae8,moveis_decoracao,alfenas,MG,4,1
98017,ffff42319e9b2d713724ae527742af25,cama_mesa_banho,taboao da serra,SP,1,2
98016,fffeda5b6d849fbd39689bb92087f431,telefonia,rio de janeiro,RJ,1,3
98015,fffecc9f79fd8c764f843e9951b11341,cama_mesa_banho,parobe,RS,1,4
98014,fffcb937e9dd47a13f05ecb8290f4d3e,livros_interesse_geral,sao paulo,SP,1,5
98013,fffb97495f78be80e2759335275df2aa,cama_mesa_banho,belo horizonte,MG,1,6
98012,fffa0238b217e18a8adeeda0669923a3,beleza_saude,sao paulo,SP,1,7
98011,fff93c1da78dafaaa304ff032abc6205,livros_interesse_geral,araraquara,SP,9,8
98010,fff906ecb75de5809be384e0f8d65e45,papelaria,coronel fabriciano,MG,1,9
98009,fff89c8ed4fcf69a823c1d149e429a0b,automotivo,goiania,GO,1,10


## Top Categorias que um cliente mais comprou

In [4]:
# Remove registros duplicados
pedidos_unicos_df = df.drop_duplicates(['product_category_name','order_purchase_timestamp', 'customer_id'])

# Encontra o número de pedidos e o número de pedidos únicos de cada categoria
itens_mais_frequentes_df = pedidos_unicos_df.groupby(['product_category_name', 'customer_city', 'customer_state', 'customer_id']).agg({'order_purchase_timestamp':'count'})
itens_mais_frequentes_df.columns=['no_pedidos']

# Reset o index convertendo o produto em coluna
itens_mais_frequentes_df.reset_index(inplace=True)

In [5]:
# Produtos com alto número de pedidos são considerados itens comprados com mais frequência

# Rank da categoria de produto por número de pedidos, no nível do cliente 

cliente_col = itens_mais_frequentes_df['customer_id']
pedidos_col = itens_mais_frequentes_df['no_pedidos'].astype(str)
itens_mais_frequentes_df['rank_popularidade'] = (cliente_col + pedidos_col).rank(method='min',ascending=False).astype(int)


# Lista dos top 20 itens mais populares
itens_mais_frequentes_df.sort_values('rank_popularidade',ascending=True).head(20)


,product_category_name,customer_city,customer_state,customer_id,no_pedidos,rank_popularidade
66158,moveis_decoracao,alfenas,MG,ffffa3172527f765de70084a7e53aae8,1,1
30286,cama_mesa_banho,taboao da serra,SP,ffff42319e9b2d713724ae527742af25,1,2
90447,telefonia,rio de janeiro,RJ,fffeda5b6d849fbd39689bb92087f431,1,3
25891,cama_mesa_banho,parobe,RS,fffecc9f79fd8c764f843e9951b11341,1,4
64220,livros_interesse_geral,sao paulo,SP,fffcb937e9dd47a13f05ecb8290f4d3e,1,5
22211,cama_mesa_banho,belo horizonte,MG,fffb97495f78be80e2759335275df2aa,1,6
16793,beleza_saude,sao paulo,SP,fffa0238b217e18a8adeeda0669923a3,1,7
63771,livros_interesse_geral,araraquara,SP,fff93c1da78dafaaa304ff032abc6205,1,8
74907,papelaria,coronel fabriciano,MG,fff906ecb75de5809be384e0f8d65e45,1,9
2798,automotivo,goiania,GO,fff89c8ed4fcf69a823c1d149e429a0b,1,10


In [6]:
top_cliente_itens_df = top_cliente_itens_df.drop(["customer_city", "customer_state"], axis=1)

In [7]:
# Merge dos Ranks de Categoria de produtos mais Vendidos e dos mais Populares
cliente_produto_rankings_df = pd.merge(top_cliente_itens_df, itens_mais_frequentes_df, how='inner',on=['customer_id','product_category_name'])

# Combine o preço unitário (preço máximo no nível do produto)

# Encontre o preço unitário de cada produto (máximo do preço considerado, pode ser necessário alterar para mediana ou média)

itens_preco_df = df.groupby(['product_category_name']).agg({'price':'max'})

itens_preco_df.reset_index(inplace=True)

In [8]:
# Isso garante que o mesmo preço unitário seja anexado ao produto adquirido por clientes diferentes 
cliente_produto_rankings_df = pd.merge(cliente_produto_rankings_df,itens_preco_df,how='left',on='product_category_name')
cliente_produto_rankings_df.columns

Index(['customer_id', 'product_category_name', 'quantidade_item',
       'rank_top_vendidos', 'customer_city', 'customer_state', 'no_pedidos',
       'rank_popularidade', 'price'],
      dtype='object')

In [9]:
#cliente_produto_rankings_df = cliente_produto_rankings_df.drop(["price_x"], axis=1)
#cliente_produto_rankings_df.columns = [col.replace("_y", "") for col in cliente_produto_rankings_df.columns]

KeyError: "['price_x'] not found in axis"

In [10]:
cliente_produto_rankings_df.columns

Index(['customer_id', 'product_category_name', 'quantidade_item',
       'rank_top_vendidos', 'customer_city', 'customer_state', 'no_pedidos',
       'rank_popularidade', 'price'],
      dtype='object')

In [11]:
# Filtra apenas as colunas 'product_category_name','rank_top_vendidos','rank_popularidade', 'customer_city', 'customer_state', 'price'
cliente_produto_rankings_df = cliente_produto_rankings_df[['product_category_name','rank_top_vendidos','rank_popularidade', 'customer_city', 'customer_state', 'price']]

In [12]:
cliente_produto_rankings_df.head()

,product_category_name,rank_top_vendidos,rank_popularidade,customer_city,customer_state,price
0,brinquedos,98019,98019,osasco,SP,1699.99
1,beleza_saude,98018,98018,itapecerica,MG,3124.00
2,bebes,98017,98017,nova venecia,ES,3899.00
3,cool_stuff,98016,98016,mendonca,MG,3109.99
4,cama_mesa_banho,98015,98015,sao paulo,SP,1999.98


In [13]:
# Salva o dataset
cliente_produto_rankings_df.to_csv('rankings_cliente_categoria.csv',index=False)

In [14]:
#Create a Pickle (.pkl) file with the Ranking dataframe
pickle.dump(cliente_produto_rankings_df, open('cliente_produto_ranking_modelo.pkl','wb'))